In [1]:
%cd ~/ipython-notebook/dqn4wirelesscontrol/
import sys
import time
import pandas as pd
pd.set_option('mode.chained_assignment', None)  # block warnings due to DataFrame value assignment
# Project modules
from sleep_control.integration import Emulation
import sleep_control.traffic_emulator
import sleep_control.traffic_server
from sleep_control.controller import QController, DummyController
import qlearning.qtable
import qlearning.qnn
from qlearning.qtable import QAgent
from qlearning.qnn import QAgentNN
from qlearning.mixin import PhiMixin

Using gpu device 0: GeForce GTX TITAN X (CNMeM is disabled, CuDNN 4007)
/usr/local/lib/python2.7/dist-packages/theano/tensor/signal/downsample.py:5: UserWarning: downsample module has been moved to the pool module.
  warnings.warn("downsample module has been moved to the pool module.")


/home/admin-326/Data/ipython-notebook/dqn4wirelesscontrol


Setting-up data

In [2]:
session_df = pd.read_csv(filepath_or_buffer='./sleep_control/data/net_traffic_nonull_sample.dat', 
                         sep=',', 
                         names=['uid','location','startTime_unix','duration_ms','domainProviders','domainTypes','domains','bytesByDomain','requestsByDomain'])
session_df.index.name = 'sessionID'
session_df['endTime_unix'] = session_df['startTime_unix'] + session_df['duration_ms']
session_df['startTime_datetime'] = pd.to_datetime(session_df['startTime_unix'], unit='ms')  # convert start time to readible date_time strings
session_df['endTime_datetime'] = pd.to_datetime(session_df['endTime_unix'], unit='ms')
session_df['totalBytes'] = session_df['bytesByDomain'].apply(lambda x: x.split(';')).map(lambda x: sum(map(float, x)))  # sum bytes across domains
session_df['totalRequests'] = session_df['requestsByDomain'].apply(lambda x: x.split(';')).map(lambda x: sum(map(float, x)))  # sum requests across domains
session_df.sort(['startTime_datetime'], ascending=True, inplace=True)  # get it sorted
session_df['interArrivalDuration_datetime'] = session_df.groupby('location')['startTime_datetime'].diff()  # group-wise diff
session_df['interArrivalDuration_ms'] = session_df.groupby('location')['startTime_unix'].diff()  # group-wise diff

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:10: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [3]:
class QAgentNNPhi(PhiMixin, QAgentNN):
    def __init__(self, **kwargs):
        super(QAgentNNPhi, self).__init__(**kwargs)

Experiment 1

In [4]:
session_df_loc = session_df[session_df['location']=='第三食堂']

In [36]:
te = sleep_control.traffic_emulator.TrafficEmulator(session_df=session_df, time_step=pd.Timedelta(days=1), verbose=2)
ts = sleep_control.traffic_server.TrafficServer(cost=5, verbose=2)
# actions = [(s, c) for s in [True, False] for c in ['serve_all', 'queue_all', 'random_serve_and_queue']]
actions = [(True, None), (False, 'serve_all')]
# agent = QAgent(actions=actions, alpha=0.5, gamma=0.5, explore_strategy='epsilon', epsilon=0.1)
# agent = QAgentNN(dim_state=(1, 1, 3), range_state=((((0, 10), (0, 10), (0, 10)),),),
#                  learning_rate=0.01, reward_scaling=10, batch_size=100, freeze_period=50, memory_size=200, num_buffer=2,
#                  actions=actions, alpha=0.5, gamma=0.5, explore_strategy='epsilon', epsilon=0.2,
#                  verbose=2
#                  )
phi_length = 10
range_state_slice = [(0, 10), (0, 10), (0, 10), (0, 1), (0, 1)]
agent = QAgentNNPhi(
        phi_length=phi_length,
        dim_state=(1, phi_length, 3+2),
        range_state=[[range_state_slice]*phi_length],
        actions=actions,
        learning_rate=0.01, reward_scaling=10, reward_scaling_update='fixed', batch_size=100,
        freeze_period=50, memory_size=200, num_buffer=2,
        alpha=0.5, gamma=0.5, explore_strategy='epsilon', epsilon=0.02,
        verbose=2)
c = QController(agent=agent)
emu = sleep_control.integration.Emulation(te=te, ts=ts, c=c)

In [41]:
# %%prun -T prun_log_9.txt -D prun_log_9.stat  # for code profiling
t = time.time()
old_stdout = sys.stdout
dt = time.localtime()
dt = "{}-{}-{}_{:2d}{:2d}".format(dt.tm_year, dt.tm_mon, dt.tm_mday, dt.tm_hour,dt.tm_min)
log_file = open("sleep_control/experiments/log/message_"+dt+".log","w")
sys.stdout = log_file
TOTAL_EPOCHS = 3600*24*0.01/2  # 7 days
# TOTAL_EPOCHS = 3600*24*1/2  # 1 day
while emu.te.epoch is not None and emu.epoch<TOTAL_EPOCHS:
    # log time
    print "Epoch {}, ".format(emu.epoch),
    left = emu.te.head_datetime + emu.te.epoch*emu.te.time_step
    right = left + emu.te.time_step
    print "{} - {}".format(left.strftime("%Y-%m-%d %H:%M:%S"), right.strftime("%Y-%m-%d %H:%M:%S"))
    emu.step()
    print
sys.stdoutd = old_stdout
time.time()-t

4.4025559425354

In [40]:
emu.te.head_datetime + emu.te.epoch*emu.te.time_step

Timestamp('2014-09-04 01:00:17.422000')

'2016-6-2_1639'

time.struct_time(tm_year=2016, tm_mon=6, tm_mday=2, tm_hour=16, tm_min=36, tm_sec=15, tm_wday=3, tm_yday=154, tm_isdst=0)

Experiment 2

In [4]:
session_df_loc = session_df[session_df['location']=='第三食堂']

In [16]:
te = sleep_control.traffic_emulator.TrafficEmulator(session_df=session_df_loc, time_step=pd.Timedelta(seconds=2), verbose=2)
ts = sleep_control.traffic_server.TrafficServer(cost=5, verbose=2)
# actions = [(s, c) for s in [True, False] for c in ['serve_all', 'queue_all', 'random_serve_and_queue']]
actions = [(True, None), (False, 'serve_all')]
# agent = QAgent(actions=actions, alpha=0.5, gamma=0.5, explore_strategy='epsilon', epsilon=0.1)
# agent = QAgentNN(dim_state=(1, 1, 3), range_state=((((0, 10), (0, 10), (0, 10)),),),
#                  learning_rate=0.01, reward_scaling=10, batch_size=100, freeze_period=50, memory_size=200, num_buffer=2,
#                  actions=actions, alpha=0.5, gamma=0.5, explore_strategy='epsilon', epsilon=0.2,
#                  verbose=2
#                  )
phi_length = 100
range_state_slice = [(0, 10), (0, 10), (0, 10), (0, 1), (0, 1)]
agent = QAgentNNPhi(
        phi_length=phi_length,
        dim_state=(1, phi_length, 3+2),
        range_state=[[range_state_slice]*phi_length],
        actions=actions,
        learning_rate=0.01, reward_scaling=10, reward_scaling_update='fixed', batch_size=100,
        freeze_period=50, memory_size=200, num_buffer=2,
        alpha=0.5, gamma=0.5, explore_strategy='epsilon', epsilon=0.02,
        verbose=2)
c = QController(agent=agent)
emu = sleep_control.integration.Emulation(te=te, ts=ts, c=c)

In [17]:
old_stdout = sys.stdout
log_file = open("sleep_control/experiments/log/message_2016-6-2_1040.log","w")
sys.stdout = log_file
TOTAL_EPOCHS = 3600*24*7/2
while emu.epoch is not None and emu.epoch<TOTAL_EPOCHS:
    # log time
    print "Epoch {}, ".format(emu.epoch),
    left = emu.te.head_datetime + emu.te.epoch*emu.te.time_step
    right = left + emu.te.time_step
    print "{} - {}".format(left.strftime("%Y-%m-%d %H:%M:%S"), right.strftime("%Y-%m-%d %H:%M:%S"))
    emu.step()
    print

Experiment 3

In [4]:
session_df_loc = session_df[session_df['location']=='第三食堂']

In [ ]:
te = sleep_control.traffic_emulator.TrafficEmulator(session_df=session_df_loc, time_step=pd.Timedelta(seconds=2), verbose=2)
ts = sleep_control.traffic_server.TrafficServer(cost=5, verbose=2)
# actions = [(s, c) for s in [True, False] for c in ['serve_all', 'queue_all', 'random_serve_and_queue']]
actions = [(True, None), (False, 'serve_all')]
# agent = QAgent(actions=actions, alpha=0.5, gamma=0.5, explore_strategy='epsilon', epsilon=0.1)
# agent = QAgentNN(dim_state=(1, 1, 3), range_state=((((0, 10), (0, 10), (0, 10)),),),
#                  learning_rate=0.01, reward_scaling=10, batch_size=100, freeze_period=50, memory_size=200, num_buffer=2,
#                  actions=actions, alpha=0.5, gamma=0.5, explore_strategy='epsilon', epsilon=0.2,
#                  verbose=2
#                  )
phi_length = 10
range_state_slice = [(0, 10), (0, 10), (0, 10), (0, 1), (0, 1)]
agent = QAgentNNPhi(
        phi_length=phi_length,
        dim_state=(1, phi_length, 3+2),
        range_state=[[range_state_slice]*phi_length],
        actions=actions,
        learning_rate=0.01, reward_scaling=10, reward_scaling_update='adaptive', batch_size=100,
        freeze_period=50, memory_size=200, num_buffer=2,
        alpha=0.5, gamma=0.5, explore_strategy='epsilon', epsilon=0.02,
        verbose=2)
c = QController(agent=agent)
emu = sleep_control.integration.Emulation(te=te, ts=ts, c=c)

In [ ]:
old_stdout = sys.stdout
log_file = open("sleep_control/experiments/log/message_2016-6-1_2330.log","w")
sys.stdout = log_file
TOTAL_EPOCHS = 3600*24*4/2
while emu.epoch is not None and emu.epoch<TOTAL_EPOCHS:
    # log time
    print "Epoch {}, ".format(emu.epoch),
    left = emu.te.head_datetime + emu.te.epoch*emu.te.time_step
    right = left + emu.te.time_step
    print "{} - {}".format(left.strftime("%Y-%m-%d %H:%M:%S"), right.strftime("%Y-%m-%d %H:%M:%S"))
    emu.step()
    print

Experiment 4

In [4]:
session_df_loc = session_df[session_df['location']=='第三食堂']

In [ ]:
te = sleep_control.traffic_emulator.TrafficEmulator(session_df=session_df_loc, time_step=pd.Timedelta(seconds=2), verbose=2)
ts = sleep_control.traffic_server.TrafficServer(cost=5, verbose=2)
# actions = [(s, c) for s in [True, False] for c in ['serve_all', 'queue_all', 'random_serve_and_queue']]
actions = [(True, None), (False, 'serve_all')]
# agent = QAgent(actions=actions, alpha=0.5, gamma=0.5, explore_strategy='epsilon', epsilon=0.1)
# agent = QAgentNN(dim_state=(1, 1, 3), range_state=((((0, 10), (0, 10), (0, 10)),),),
#                  learning_rate=0.01, reward_scaling=10, batch_size=100, freeze_period=50, memory_size=200, num_buffer=2,
#                  actions=actions, alpha=0.5, gamma=0.5, explore_strategy='epsilon', epsilon=0.2,
#                  verbose=2
#                  )
phi_length = 10
range_state_slice = [(0, 10), (0, 10), (0, 10), (0, 1), (0, 1)]
agent = QAgentNNPhi(
        phi_length=phi_length,
        dim_state=(1, phi_length, 3+2),
        range_state=[[range_state_slice]*phi_length],
        actions=actions,
        learning_rate=0.01, reward_scaling=10, reward_scaling_update='adaptive', batch_size=100,
        freeze_period=50, memory_size=200, num_buffer=2,
        alpha=0.5, gamma=0.5, explore_strategy='epsilon', epsilon=0.02,
        verbose=2)
c = QController(agent=agent)
emu = sleep_control.integration.Emulation(te=te, ts=ts, c=c)

In [ ]:
old_stdout = sys.stdout
log_file = open("sleep_control/experiments/log/message_2016-6-1_2400.log","w")
sys.stdout = log_file
TOTAL_EPOCHS = 3600*24*4/2
while emu.epoch is not None and emu.epoch<TOTAL_EPOCHS:
    # log time
    print "Epoch {}, ".format(emu.epoch),
    left = emu.te.head_datetime + emu.te.epoch*emu.te.time_step
    right = left + emu.te.time_step
    print "{} - {}".format(left.strftime("%Y-%m-%d %H:%M:%S"), right.strftime("%Y-%m-%d %H:%M:%S"))
    emu.step()
    print

In [7]:
reload(sleep_control.integration)
reload(sleep_control.traffic_server)
reload(sleep_control.traffic_emulator)
import qlearning
reload(qlearning.qnn)
reload(qlearning.qtable)

<module 'qlearning.qtable' from 'qlearning/qtable.pyc'>